# Chapter 4

## Problem 4.1
(1) 检验 $H_0: \mu = \mu_0 = (4,50,10)'$

In [93]:
import numpy as np
import pandas as pd
from scipy import stats

path = "/Users/xinby/Desktop/Sufe/Multivariate-Stat-Analysis/Chpt4/exec4.1.xlsx"
dat1 = pd.read_excel(path)
dat1 = dat1.values
print(dat1)

[[ 3.7 48.5  9.3]
 [ 5.7 65.1  8. ]
 [ 3.8 47.2 10.9]
 [ 3.2 53.2 12. ]
 [ 3.1 55.5  9.7]
 [ 4.6 36.1  7.9]
 [ 2.4 24.8 14. ]
 [ 7.2 33.1  7.6]
 [ 6.7 47.4  8.5]
 [ 5.4 54.1 11.3]
 [ 3.9 36.9 12.7]
 [ 4.5 58.8 12.3]
 [ 3.5 27.8  9.8]
 [ 4.5 40.2  8.4]
 [ 1.5 13.5 10.1]
 [ 8.5 56.4  7.1]
 [ 4.5 71.6  8.2]
 [ 6.5 52.8 10.9]
 [ 4.1 44.1 11.2]
 [ 5.5 40.9  9.4]]


/Users/xinby/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


由于没有给出总体方差，需要用样本协方差矩阵代替总体协方差矩阵，公式：$T^2 = n(\bar x - \mu_0)'S^{-1}(\bar x -\mu_0)$，当原假设成立时有：$\frac{n-p}{p(n-1)}T^2\sim F(p,n-p)$
  
**[Coding Tips]**
- 在 `numpy` 中，` np.cov()` 可以用来计算协方差（协方差矩阵），其中参数设置`rowvar = False` 会将每一列认为是一个变量，这一点需要特殊注意，其默认值*True*与一般统计习惯不符
- 保持良好的计算习惯：这里我们需要计算$S^{-1}(\bar x -\mu_0)$，因此我们应当通过求解方程（设t为未知向量）$St=(\bar x-\mu_0$来避免显式求解逆运算，在python的实现为`np.linalg.solve(S, xbar-mu0)`
- `scipy.stats.f`包中，`dfn`与`dfd`分别代表第1、2自由度，不要弄反($F(dfn,dfd)$)，`stats.f.sf`相当于$1-cdf$，但可能具体数值计算上会稍微更精准一些

In [160]:
n = len(dat1)
xbar = np.average(dat1,axis=0)
print("xbar:\n",xbar)
mu0 = [4,50,10]
# S as Sample Covariance Matrix
S = np.cov(dat1,rowvar=False) 
print("S:\n",S)
# T_sq as Hotelling T
T_sq = n*(xbar - mu0).transpose().dot(np.linalg.solve(S, xbar-mu0)) # T_sq is Hotelling T
print("Tsquare:\n",T_sq)
# From Hotelling T_sq to F 
p = dat1.shape[1] # p is the num of variables (width of the matrix)
F = (n-p)/(p*(n-1))*T_sq
print("F:\n",F)
# From F~F(p,n-p) compute p_value
alpha = 0.05
df1 = p
df2 = n-p
pvalue = stats.f.sf(x = F, dfn = df1, dfd = df2)
print("p_value:\n",pvalue)

xbar:
 [ 4.64  45.4    9.965]
S:
 [[  2.87936842  10.01        -1.80905263]
 [ 10.01       199.78842105  -5.64      ]
 [ -1.80905263  -5.64         3.62765789]]
Tsquare:
 9.738772855435641
F:
 2.904546290217647
p_value:
 0.0649283353805298


由上述输出可知，p略大于0.05，无法拒绝原假设，可以认为与给定均值统计意义上相等。



## Prob 4.3
在多元正态和两总体协方差矩阵相等的假定下甲和乙两种品牌轮胎的耐用性指标是否有显著不同($\alpha=0.05$)？

In [201]:
path = "/Users/xinby/Desktop/Sufe/Multivariate-Stat-Analysis/Chpt4/exec4.3.xlsx"
dat3 = pd.read_excel(path)
dat3_1 = dat3[dat3["g"]=="甲"].drop(columns='g').values
dat3_2 = dat3[dat3["g"]=="乙"].drop(columns='g').values
print("dat3.1:\n",dat3_1,"\ndat3.2:\n",dat3_2)

dat3.1:
 [[194 192 141]
 [208 188 165]
 [233 217 171]
 [241 222 201]
 [265 252 207]
 [269 283 191]] 
dat3.2:
 [[239 127  90]
 [189 105  85]
 [224 123  79]
 [243 123 110]
 [243 117 100]
 [226 125  75]]


/Users/xinby/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


对于两独立样本均值的比较推断，在本假设中有正态、协方差相等，故可以用如下公式：
$$T^2 = (1/n_1+1/n_2)^{-1}(\bar x-\bar y)'S_p^{-1}(\bar x-\bar y)$$
当原假设为真，则有：
$$(n_1+n_2-p-1)/(p(n_1+n_2-2))T^2\sim F(p,n_1+n_2-p-1)$$

In [226]:
n1 = len(dat3_1)
n2 = len(dat3_2)
p = dat3.shape[1]
print("n1,n2,p:",n1,n2,p)

xbar = dat3_1.mean(axis =0)
ybar = dat3_2.mean(axis =0)
S1 = np.cov(dat3_1,rowvar=False)
S2 = np.cov(dat3_2,rowvar=False)
Sp = ((n1-1)*S1 + (n2-1)*S2)/(n1+n2-2)
print("xbar:\n",xbar,"\nybar:\n",ybar)
print("S1:\n",S1,"\nS2:\n",S2,"\nSp:\n",Sp)

T_sq = (n1*n2)/(n1+n2)*(xbar - ybar).transpose().dot(np.linalg.solve(Sp,xbar-ybar))
F = (n1+n2-p-1)/(p*(n1+n2-2))*T_sq
print("T_sq:\n",T_sq,"\nF:\n",F)

df1 = p
df2 = n1+n2-p-1 
stats.f.sf(F,dfn = df1,dfd = df2)




n1,n2,p: 6 6 4
xbar:
 [235.         225.66666667 179.33333333] 
ybar:
 [227.33333333 120.          89.83333333]
S1:
 [[ 901.2        1026.4         666.4       ]
 [1026.4        1324.26666667  644.13333333]
 [ 666.4         644.13333333  623.06666667]] 
S2:
 [[421.86666667 128.         143.66666667]
 [128.          65.2         -0.6       ]
 [143.66666667  -0.6        174.16666667]] 
Sp:
 [[661.53333333 577.2        405.03333333]
 [577.2        694.73333333 321.76666667]
 [405.03333333 321.76666667 398.61666667]]
T_sq:
 365.3141334419115 
F:
 63.92997335233451


1.3606163393532304e-05